# Some plot code taken out from the working NB

In [8]:
brush1 = alt.selection(type="interval", encodings=['x'])
brush2 = alt.selection(type="interval", encodings=['x'])


scatter = alt.Chart(MPsedMelted).mark_point().encode(
    x = 'freq_sum:Q',
    y = 'conc_sum:Q',
    tooltip = 'sample_name:N'
).transform_filter(
    brush1 | brush2
).transform_aggregate(
    conc_sum = 'sum(conc)',
    freq_sum = 'sum(freq)',
    sample_name = 'values(sample)',
    groupby = ['sample']
# ).transform_calculate(
#     b1l='(isDefined(brush1.x) ? (brush1.x[0]) : 1)',
#     b1u='(isDefined(brush1.x) ? (brush1.x[1]) : 1)',
#     b2l='(isDefined(brush2.x) ? (brush2.x[0]) : 1)',
#     b2u='(isDefined(brush2.x) ? (brush2.x[1]) : 1)',
# ).transform_window(
#     conc_sum = 'sum(conc)',
#     frame = [brush1.x[0],brush1.x[1]],
#     groupby=['sample']
# ).transform_window(
#     freq_sum = 'sum(freq)',
#     frame = [brush2.x[0],brush2.x[1]],
#     groupby=['sample']
)


RegLine = scatter.transform_regression(
    'freq_sum', 'conc_sum', method="linear",
).mark_line(
    color="red"
)


RegParams = scatter.transform_regression(
    'freq_sum', 'conc_sum', method="linear", params=True
).mark_text(align='left', lineBreak='\n').encode(
    x=alt.value(120),  # pixels from left
    y=alt.value(20),  # pixels from top
    text='params:N'
).transform_calculate(
    params='"r² = " + round(datum.rSquared * 100)/100 + \
    "      y = " + round(datum.coef[1] * 10)/10 + "x" + " + " + round(datum.coef[0] * 10)/10'
)


MP = alt.Chart(MPsedMelted).mark_line().encode(
    x = alt.X('lower', scale=alt.Scale(type='linear')),
    y = 'mean(conc)'
).add_selection(
    brush1
).properties(
    height = 100
)


sed = alt.Chart(MPsedMelted).mark_line().encode(
    x = alt.X('lower', scale=alt.Scale(type='linear')),
    y = 'mean(freq)'
).add_selection(
    brush2
).properties(
    height = 100
)


(scatter + RegLine + RegParams) & alt.vconcat(MP, sed)

alt.VConcatChart(...)

In [2]:
import altair as alt
from vega_datasets import data
cars = data.cars.url

brush1 = alt.selection(type="interval", encodings=['x'])
brush2 = alt.selection(type="interval", encodings=['x'])


scatter = alt.Chart(cars).mark_point().encode(
    x = 'HP_sum:Q',
    y = 'Dis_sum:Q',
    tooltip = 'Origin:N'
).transform_filter(  # Ok, I can filter the whole data set, but that always acts on both variables (HP and displacement) together... -> not what I want.
    brush1 | brush2
).transform_aggregate(
    Dis_sum = 'sum(Displacement)',
    HP_sum = 'sum(Horsepower)',
    groupby = ['Origin', 'HP_sum']
# ).transform_calculate(  # Can I extract the selection boundaries like that? And if yes: how can I use these extracts to calculate the aggregationsof HP and displacement?
#     b1_lower='(isDefined(brush1.x) ? (brush1.x[0]) : 1)',
#     b1_upper='(isDefined(brush1.x) ? (brush1.x[1]) : 1)',
#     b2_lower='(isDefined(brush2.x) ? (brush2.x[0]) : 1)',
#     b2_upper='(isDefined(brush2.x) ? (brush2.x[1]) : 1)',
# ).transform_window(  # Maybe instead of calculate I can use two window transforms...??
#     conc_sum = 'sum(conc)',
#     frame = [brush1.x[0],brush1.x[1]],  # This will not work, as it sets the frame relative (back- and foreward) to each datum (i.e. sliding window), I need it to correspond to the entire data set
#     groupby=['sample']
# ).transform_window(
#     freq_sum = 'sum(freq)',
#     frame = [brush2.x[0],brush2.x[1]],  # ...same problem here
#     groupby=['sample']
)

range_sel1 = alt.Chart(cars).mark_line().encode(
    x = 'Year:T',
    y = 'mean(Horsepower):Q'
).add_selection(
    brush1
).properties(
    height = 100
)

range_sel2 = alt.Chart(cars).mark_line().encode(
    x = 'Year:T',
    y = 'mean(Displacement):Q'
).add_selection(
    brush2
).properties(
    height = 100
)


scatter & range_sel1 & range_sel2

alt.VConcatChart(...)